# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies
from census import Census
import hvplot.pandas
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt
from pathlib import Path

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vorgashor,67.5833,63.9500,1.58,96,100,13.41,RU,1727767221
1,1,waitangi,-43.9535,-176.5597,10.46,88,89,0.89,NZ,1727767223
2,2,puerto natales,-51.7236,-72.4875,1.93,98,100,2.31,CL,1727767225
3,3,moberly,39.4184,-92.4382,19.99,68,0,4.63,US,1727767227
4,4,adamstown,-25.0660,-130.1015,19.14,68,44,4.58,PN,1727767229


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size="Humidity",
    geo = True,
    tiles = "OSM",
    color= "City",
    alpha=0.6,        
    hover_cols=['City', 'Humidity']
)
# Display the map plot_1
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= 21) &
    (city_data_df['Max Temp'] <= 27) &
    (city_data_df['Cloudiness'] == 0) &
    (city_data_df['Wind Speed'] <= 4.5)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display the filtered DataFrame
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
57,57,ghadamis,30.1337,9.5007,25.49,30,0,4.34,LY,1727767336
59,59,gumdag,39.2061,54.5906,24.37,15,0,3.85,TM,1727767340
92,92,metlili chaamba,32.2667,3.6333,22.52,60,0,2.06,DZ,1727767412
95,95,mermoz boabab,14.7065,-17.4758,26.88,89,0,2.06,SN,1727767418
113,113,forio,40.7342,13.8646,21.35,82,0,1.79,IT,1727767454
114,114,makaha,21.4694,-158.2175,26.63,69,0,0.00,US,1727767456
126,126,oltinko'l,40.8011,72.1634,25.57,20,0,1.30,UZ,1727767480
130,130,crane,31.3974,-102.3501,22.92,29,0,3.23,US,1727767488
208,208,awjilah,29.1081,21.2869,22.51,52,0,3.48,LY,1727767650
209,209,bariri,-22.0744,-48.7403,21.94,48,0,1.84,BR,1727767652


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
57,ghadamis,LY,30.1337,9.5007,30,
59,gumdag,TM,39.2061,54.5906,15,
92,metlili chaamba,DZ,32.2667,3.6333,60,
95,mermoz boabab,SN,14.7065,-17.4758,89,
113,forio,IT,40.7342,13.8646,82,
114,makaha,US,21.4694,-158.2175,69,
126,oltinko'l,UZ,40.8011,72.1634,20,
130,crane,US,31.3974,-102.3501,29,
208,awjilah,LY,29.1081,21.2869,52,
209,bariri,BR,-22.0744,-48.7403,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:

# Set parameters to search for a hotel
radius = 10000
params = {  
    'categories': 'accommodation.hotel',
    'limit': 1,  # Get the first result
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ghadamis - nearest hotel: فندق عين الفارس
gumdag - nearest hotel: No hotel found
metlili chaamba - nearest hotel: No hotel found
mermoz boabab - nearest hotel: La maison blanche
forio - nearest hotel: Hotel Terme Colella
makaha - nearest hotel: No hotel found
oltinko'l - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
awjilah - nearest hotel: No hotel found
bariri - nearest hotel: No hotel found
hassi messaoud - nearest hotel: AMC
buka - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
castaic - nearest hotel: Days Inn
aktau - nearest hotel: Гульдана
santa cruz del sur - nearest hotel: No hotel found
pacific grove - nearest hotel: Pacific Grove Inn
hun - nearest hotel: فندق الهروج
saint-pierre - nearest hotel: Tropic Hotel
comandante fontana - nearest hotel: No hotel found
tash-kumyr - nearest hotel: Гостиница Таш-Кумыр
tamanrasset - nearest hotel: فندق الأمان
mahina - nearest hotel: Motu Martin
sirte - nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
57,ghadamis,LY,30.1337,9.5007,30,فندق عين الفارس
59,gumdag,TM,39.2061,54.5906,15,No hotel found
92,metlili chaamba,DZ,32.2667,3.6333,60,No hotel found
95,mermoz boabab,SN,14.7065,-17.4758,89,La maison blanche
113,forio,IT,40.7342,13.8646,82,Hotel Terme Colella
114,makaha,US,21.4694,-158.2175,69,No hotel found
126,oltinko'l,UZ,40.8011,72.1634,20,No hotel found
130,crane,US,31.3974,-102.3501,29,No hotel found
208,awjilah,LY,29.1081,21.2869,52,No hotel found
209,bariri,BR,-22.0744,-48.7403,48,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    size="Humidity",
    geo = True,
    tiles = "OSM",
    color= "City",
    alpha=0.6,        
    hover_cols=['City','Humidity','Hotel Name','Country'] 
)
# Display the map plot_1
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)